# Import the libraries and load dataset

In [1]:
import numpy as np
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local').appName('bank_marketing_data_cleaning').getOrCreate()

df = spark.read.options(header=True, inferSchema=True).csv('csv_files/bank_marketing.csv')
df.show(10)
df.printSchema()
df.count()

+---------+---+-----------+-------+-------------------+--------------+-------+----+---------+-----+---+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|client_id|age|        job|marital|          education|credit_default|housing|loan|  contact|month|day|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---------+---+-----------+-------+-------------------+--------------+-------+----+---------+-----+---+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|        0| 56|  housemaid|married|           basic.4y|            no|     no|  no|telephone|  may| 13|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
|        1| 57|   services|married|        high.school|       unknown|     no|  no|telephone|  may| 19|     149|       1|  999|     

41188

# Data cleaning and rearranging

In [3]:
for c in df.columns:
    num_unknown = df.filter(f'{c} == "unknown"').count()
    print(f'Num of unknown in {c}: {num_unknown}')

Num of unknown in client_id: 0
Num of unknown in age: 0


Num of unknown in job: 330
Num of unknown in marital: 80
Num of unknown in education: 1731
Num of unknown in credit_default: 8597
Num of unknown in housing: 990
Num of unknown in loan: 990
Num of unknown in contact: 0
Num of unknown in month: 0
Num of unknown in day: 0
Num of unknown in duration: 0
Num of unknown in campaign: 0
Num of unknown in pdays: 0
Num of unknown in previous: 0
Num of unknown in poutcome: 0
Num of unknown in emp_var_rate: 0
Num of unknown in cons_price_idx: 0
Num of unknown in cons_conf_idx: 0
Num of unknown in euribor3m: 0
Num of unknown in nr_employed: 0
Num of unknown in y: 0


In [4]:
for c in ['job', 'marital', 'education']:
    df = df.withColumn(c, when(col(c) == 'unknown', None).otherwise(col(c)))

for c in ['credit_default', 'housing', 'loan', 'y']:
    df = df.withColumn(c, when(col(c) == 'yes', 1).otherwise(0))

df = df.withColumn('poutcome', when(col('poutcome') == 'success', 1).otherwise(0))

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---+---+-------+---------+--------------+-------+----+-------+-----+---+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|client_id|age|job|marital|education|credit_default|housing|loan|contact|month|day|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---------+---+---+-------+---------+--------------+-------+----+-------+-----+---+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|        0|  0|330|     80|     1731|             0|      0|   0|      0|    0|  0|       0|       0|    0|       0|       0|           0|             0|            0|        0|          0|  0|
+---------+---+---+-------+---------+--------------+-------+----+-------+-----+---+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+



In [5]:
df = df.dropna(subset=['job', 'marital'])

education_mode = df.groupBy('education').count().orderBy(desc('count')).select('education').collect()[0]
education_mode = (education_mode['education'])
df = df.fillna(value=education_mode, subset=['education'])
df.groupBy('education').count().orderBy(desc('count')).show()

+-------------------+-----+
|          education|count|
+-------------------+-----+
|  university.degree|13692|
|        high.school| 9464|
|           basic.9y| 6006|
|professional.course| 5225|
|           basic.4y| 4118|
|           basic.6y| 2264|
|         illiterate|   18|
+-------------------+-----+



In [6]:
month_map = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06', 
             'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}

df = df.na.replace(month_map, None, 'month')

df.groupBy('month').count().orderBy('month').show()

+-----+-----+
|month|count|
+-----+-----+
|   03|  541|
|   04| 2620|
|   05|13619|
|   06| 5245|
|   07| 7118|
|   08| 6108|
|   09|  562|
|   10|  707|
|   11| 4087|
|   12|  180|
+-----+-----+



In [7]:
df = df.withColumn('year', lit('2022'))
df = df.withColumn('day', df.day.cast('string'))

df = df.withColumn('date', concat_ws('-', 'year', 'month', 'day'))

In [8]:
df = df.withColumn('last_contact_date', to_date('date'))
df = df.drop('year', 'month', 'day', 'date')

# Split into 3 DataFrames: client, campaign, economics

In [9]:
client = df.select('client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan')
client = client.withColumnRenamed('client_id', 'id')
client.show(10)


campaign = df.select('client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'last_contact_date')
campaign = campaign.withColumnRenamed('campaign', 'number_contacts') \
                   .withColumnRenamed('duration', 'contact_duration') \
                   .withColumnRenamed('previous', 'previous_campaign_contacts') \
                   .withColumnRenamed('poutcome', 'previous_outcome') \
                   .withColumnRenamed('y', 'campaign_outcome')
campaign.show(10)

economics = df.select('client_id', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed')
economics = economics.withColumnRenamed('euribor3m', 'euribor_three_months') \
                     .withColumnRenamed('nr_employed', 'number_employed')
economics.show(10)

+---+---+-----------+-------+-------------------+--------------+-------+----+
| id|age|        job|marital|          education|credit_default|housing|loan|
+---+---+-----------+-------+-------------------+--------------+-------+----+
|  0| 56|  housemaid|married|           basic.4y|             0|      0|   0|
|  1| 57|   services|married|        high.school|             0|      0|   0|
|  2| 37|   services|married|        high.school|             0|      1|   0|
|  3| 40|     admin.|married|           basic.6y|             0|      0|   0|
|  4| 56|   services|married|        high.school|             0|      0|   1|
|  5| 45|   services|married|           basic.9y|             0|      0|   0|
|  6| 59|     admin.|married|professional.course|             0|      0|   0|
|  7| 41|blue-collar|married|  university.degree|             0|      0|   0|
|  8| 24| technician| single|professional.course|             0|      1|   0|
|  9| 25|   services| single|        high.school|             0|

# Save as csv

In [10]:
client.write.option('header', True).csv('csv_files/client.csv')
campaign.write.option('header', True).csv('csv_files/campaign.csv')
economics.write.option('header', True).csv('csv_files/economics.csv')